<h1> <b> Michelin restaurants in Italy Web scraping  </b> <img src="https://styles.redditmedia.com/t5_2s8x6/styles/communityIcon_ftfh5okvyxj01.png" width=120 style="vertical-align: middle"> </h1>

In today's world, people are more eager than ever to discover culinary experiences that are both unique and unforgettable. The culinary arts have evolved from a necessity to a refined skill, with passionate chefs pushing the boundaries of creativity and flavor. For food lovers, travelers, and students of gastronomy, resources like the Michelin Guide are invaluable. This platform, accessible via the <a href="https://guide.michelin.com/en/it/restaurants" > Michelin Guide website</a>, provides information, reviews, and ratings for restaurants throughout Italy and beyond, recognized for their quality, innovation, and exceptional culinary experiences.

Our team is dedicated to creating a search engine tailored for food lovers, helping users discover and rank Michelin-starred restaurants across Italy based on their unique preferences. Your mission: to create an efficient and intuitive tool to explore the best culinary experiences in Italy.

<h1> <b> Import Libraries </b> <img src="https://preview.redd.it/snoovatar/avatars/nftv2_bmZ0X2VpcDE1NToxMzdfZWI5NTlhNzE1ZGZmZmU2ZjgyZjQ2MDU1MzM5ODJjNDg1OWNiMTRmZV8yMTQ1NzYzNg_rare_46f1cdb1-634f-4c1d-8344-2be06c7880d4-headshot.png?width=256&height=256&crop=smart&auto=webp&s=400ead9440c7a9f06ca4c44953f24c5b765c4aac" width=150 style="vertical-align: middle"> </h1>

In [ ]:
pip install beautifulsoup4 requests pandas tqdm nltk opencage folium IPython ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import re
import time
from tqdm import tqdm
import math
import pickle
import heapq
import nltk
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# <h1> <b> 1.0.0 Data Collection </b> <img src="https://styles.redditmedia.com/t5_88fouv/styles/profileIcon_snoo-nftv2_bmZ0X2VpcDE1NToxMzdfZWI5NTlhNzE1ZGZmZmU2ZjgyZjQ2MDU1MzM5ODJjNDg1OWNiMTRmZV84MTg4MjQz_rare_177d88d7-3b80-4acf-b709-ba3f58c39a52-headshot.png" width=150 style="vertical-align: middle"> </h1>

In [ ]:
# Sends GET request to Michelin Guide website
response = requests.get("https://guide.michelin.com/en/it/restaurants")

In [ ]:
soup = BeautifulSoup(response.content, features="lxml")

# Display all first 1000 characters of html
soup.prettify()[:1000]

'<!DOCTYPE html>\n<html class="full-screen-mobile" dir="" lang="en-US">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1.0, user-scalable=0" name="viewport"/>\n  <meta content="" name="author"/>\n  <meta content="#fff" name="theme-color"/>\n  <meta content="MICHELIN Guide" property="og:site_name"/>\n  <meta content="Italy MICHELIN Restaurants – The MICHELIN Guide" itemprop="name"/>\n  <meta content="Italy MICHELIN Restaurants – The MICHELIN Guide" property="og:title"/>\n  <meta content="index, follow" name="robots"/>\n  <meta content="Article" property="og:type"/>\n  <meta content="https://guide.michelin.com/en/it/restaurants" property="og:url"/>\n  <meta content="Starred restaurants, Bib Gourmand and all the Restaurants of The MICHELIN Guide Italy. MICHELIN inspector reviews and insights" name="description"/>\n  <meta content="Starred restaurants, Bib Gourmand and all the Restaurants of The MICHELIN Guide Italy. MICHELIN inspector reviews and i

## 1.1. Get the list of Michelin restaurants

#### Our task is to collect the **URL** associated with each restaurant

In [ ]:
# Define the base URLs for the Michelin guide page

full_base_url = "https://guide.michelin.com"  # Full base url for constructing restaurant links
base_url = "https://guide.michelin.com/en/it/restaurants"  # Base url for restaurant listings

# Initialize an empty list to store restaurant links
restaurant_links = []

# Loop through pages 1 to 100
for page_num in tqdm(range(1, 101), desc="Reading pages", unit="page", colour="blue"):

    url = f"{base_url}/page/{page_num}"

    # Send a get request to fetch the content of the page
    response = requests.get(url)

    # Parse the page content using BeautifulSoup with the lxml parser
    soup = BeautifulSoup(response.content, features="lxml")

    links = soup.find_all('a', class_="link") # Finds all anchor tags with class 'link'

    # Extract and filter restaurant URLs from the 'href' attribute
    for link in links:
        href = link.get('href')
        if "/restaurant/" in href:
            restaurant_links.append(full_base_url + href)

    # Pause for 2 seconds to avoid overwhelming the server
    time.sleep(2)

Reading pages: 100%|██████████| 100/100 [04:32<00:00,  2.72s/page]


In [ ]:
#Let's check the number of restaurant

print(f"The number of restaurant links are : {len(restaurant_links)}")

The number of restaurant links are : 1981


In [ ]:
#Visualize the first five restaurant link:

print(*restaurant_links[:5],sep="\n")   # * is used to unpack the elements of list

https://guide.michelin.com/en/campania/gragnano/restaurant/o-me-o-il-mare
https://guide.michelin.com/en/abruzzo/popoli_1845563/restaurant/donevandro
https://guide.michelin.com/en/piemonte/alba/restaurant/ape-vino-e-cucina
https://guide.michelin.com/en/campania/sorrento/restaurant/da-bob-cook-fish
https://guide.michelin.com/en/basilicata/matera/restaurant/da-mo


## 1.2. Crawl Michelin restaurant pages
- ### Download the HTML corresponding to each of the collected URLs.
- ### After collecting each page, immediately save its HTML in a file.
- ### Organize the downloaded HTML pages into folders. Each folder will contain the HTML of the restaurants from page 1, page 2, ... of the Michelin restaurant list.

In [ ]:
# Define the base folder path for saving all restaurant HTML files for each page
base_save_path  = "Michelin Restaurants"

# Create the folder if it doesn’t already exist
os.makedirs(base_save_path , exist_ok=True)

In [ ]:
for i in range(0, len(restaurant_links), 20):
    # Calculate the folder number based on the current range index
    folder_number = (i // 20) + 1

    # Create the folder to save the HTML files for this batch of urls
    folder_name = os.path.join(base_save_path, f"page {folder_number}")
    os.makedirs(folder_name, exist_ok=True)

    # Extract the group of URLs to download for this folder
    links_to_download = restaurant_links[i:i + 20]

    # Download each URL and save the HTML content
    for idx, url in enumerate(links_to_download):
        # Generate a filename for each restaurant's HTML file
        filename = os.path.join(folder_name, f"restaurant{i + idx + 1}.html")

        # Send a GET request to fetch the content of the restaurant url
        response = requests.get(url)

        # Save the HTML content of the restaurant into the file
        with open(filename, "w", encoding="utf-8") as file:
            file.write(response.text)


## 1.3 Parse downloaded pages

### At this point, we have all the HTML documents about the restaurant of interest, and we can start to extract specific information. The list of the information we desire for each restaurant and their format is :

- #### **Restaurant Name** (to save as `restaurantName`): string;
- #### **Address** (to save as `address`): string;
- #### **City** (to save as `city`): string;
- #### **Postal Code** (to save as `postalCode`): string;
- #### **Country** (to save as `country`): string;
- #### **Price Range**** (to save as `priceRange`): string;
- #### **Cuisine Type** (to save as `cuisineType`): string;
- #### **Description** (to save as `description`): string;
- #### **Facilities and Services** (to save as `facilitiesServices`): list of strings;
- #### **Accepted Credit Cards** (to save as `creditCards`): list of strings;
- #### **Phone Number** (to save as `phoneNumber`): string;
- #### **URL** to the **Restaurant Page** (to save as `website`): string.

In [ ]:
def extract_restaurant_data(file_path):
    # Load and parse the HTML file
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, "html.parser")

    # Initialize the restaurant_info dictionary with default values
    restaurant_info = {
        'restaurantName': None,
        'address': None,
        'city': None,
        'region': None,
        'postalCode': None,
        'country': None,
        'priceRange': None,
        'cuisineType': None,
        'description': None,
        'facilitiesServices': [],
        'creditCards': [],
        'phoneNumber': None,
        'website': None
    }

    # Extract Restaurant Name
    title_tag = soup.find('h1', class_='data-sheet__title')
    if title_tag:
        restaurant_info['restaurantName'] = title_tag.get_text(strip=True)
    # Extract Region
    restaurant_info["region"] = soup.find_all(class_="breadcrumb-item")[2].text
    # Extract address Components
    address_text_tag = soup.find("div", class_="data-sheet__block--text")
    address_text = address_text_tag.get_text(strip=True)
    address_text=re.sub(r'\bloc\.\s*[^,]*,', '', address_text) # Remove location (loc) and the next part up to the comma
    if address_text:
        address_parts = address_text.replace('\n', ' ').split(",")
        restaurant_info["address"] = address_parts[0] if len(address_parts) > 0 else None
        restaurant_info["city"] = address_parts[1] if len(address_parts) > 1 else None
        restaurant_info["postalCode"] = address_parts[2] if len(address_parts) > 2 else None
        restaurant_info["country"] = address_parts[3] if len(address_parts) > 3 else None

    # Extract Price Range and Cuisine Type
    info_divs = soup.find_all("div", class_="data-sheet__block--text")
    if len(info_divs) > 1:
        info_text = info_divs[1].text.strip()
        if '·' in info_text:
            info_parts = info_text.split('·')
            restaurant_info['priceRange'] = info_parts[0].strip()
            if len(info_parts) > 1:
                restaurant_info['cuisineType'] = info_parts[1].strip()

    # Extract Description
    description_tag = soup.select_one('.data-sheet__description')
    if description_tag:
        restaurant_info['description'] = description_tag.get_text(strip=True)

    # Extract Facilities and Services
    facilities_services = [service.get_text(strip=True) for service in soup.select('.restaurant-details__services li')]
    if facilities_services:
        restaurant_info['facilitiesServices'] = facilities_services

    # Extract Accepted Credit Cards (Modify this part to return a list of credit cards)
    cards=[]
    credit_cards = soup.find_all("img", class_="lazy", height="32")  # Modify as per the HTML structure
    for img in credit_cards:
        src = img.get('src', '') or img.get('data-src', '')
        if src:# Check if 'src' is not empty
            text=src.split("/")[-1]
            cards.append(text.split("-")[0].capitalize())
    restaurant_info["creditCards"]=cards

    # Extract Phone Number
    phone_span = soup.select_one('.collapse__block-item span')
    if phone_span:
        restaurant_info['phoneNumber'] = phone_span.get_text(strip=True)

    # Extract Website URL
    website_link = soup.select_one('.collapse__block-item.link-item a')
    if website_link:
        restaurant_info['website'] = website_link['href']

    return restaurant_info

#### Let's define a sorting method to apply to the `sorted` function to correctly sort the folders of each page

In [ ]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

#### We define a `parse_all_folders` function to create a dataset composed of the specific information of each html for each restaurant of each page

In [ ]:
def parse_all_folders(base_folder):

    """ Parse all HTML files in a folder structure and aggregate into a DataFrame."""

    data = []  # List to store all parsed data

    # Traverse through each folder and file in the base_folder
    for folder_name in tqdm(sorted(os.listdir(base_folder),key=natural_sort_key),desc="Read folder of each page",unit="page"):
        folder_path = os.path.join(base_folder, folder_name)

        # Check if the path is a directory
        if os.path.isdir(folder_path):
            for filename in sorted(os.listdir(folder_path),key=natural_sort_key):
                # Only process HTML files
                if filename.endswith(".html"):
                    file_path = os.path.join(folder_path, filename)
                    # Parse the file and add to data list
                    restaurant_data = extract_restaurant_data(file_path)
                    data.append(restaurant_data)

    # Create DataFrame from collected data
    df = pd.DataFrame(data)
    return df

In [ ]:
# Usage :
base_save_path  = "Michelin Restaurants"
data = parse_all_folders(base_save_path)

# Save the DataFrame to a CSV file
data.to_csv("all_restaurants_data.csv", index=False)

Read folder of each page: 100%|██████████| 100/100 [02:57<00:00,  1.78s/page]


In [ ]:
data.head()

,restaurantName,address,city,region,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,O Me O Il Mare,Via Roma 45/47,Gragnano,Campania,80054,Italy,€€€€,"Italian Contemporary, Modern Cuisine","Known around the world as the town of pasta, G...","[Air conditioning, Interesting wine list, Whee...","[Amex, Dinersclub, Mastercard, Visa]",+39 081 620 0550,http://omeoilmare.com
1,Donevandro,via Garibaldi 2,Popoli,Abruzzo,65026,Italy,€€,"Contemporary, Seasonal Cuisine","Up until a few years ago, the owner-chef at th...",[Air conditioning],"[Mastercard, Visa]",+39 388 887 6858,http://www.donevandroristorante.it
2,Ape Vino e Cucina,Piazza Risorgimento 3,Alba,Piedmont,12051,Italy,€€,"Piedmontese, Contemporary",This attractive restaurant in the heart of Alb...,"[Air conditioning, Terrace, Wheelchair access]","[Amex, Dinersclub, Maestrocard, Mastercard, Visa]",+39 0173 363453,https://www.apewinebar.it/alba/
3,Da Bob Cook Fish,largo Parsano vecchio 16,Sorrento,Campania,80067,Italy,€€,Seafood,Working in partnership with the nearby fishmon...,"[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 081 1778 3873,https://www.dabobcookfish.com/
4,DA_MÓ,Via Bruno Buozzi 20,Matera,Basilicata,75100,Italy,€€,"Regional Cuisine, Contemporary","This new, restored restaurant in the upper par...","[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 0835 686548,https://www.damoristorante.it/


In [ ]:
data.tail()

,restaurantName,address,city,region,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
1976,Umami,Via Ugo Secondo Partigiano 1,Badalucco,Liguria,18010,Italy,€€,Modern Cuisine,A young chef with experience in renowned resta...,"[Terrace, Wheelchair access]","[Amex, Mastercard, Visa]",+39 331 338 6005,https://www.umamirestaurant.it/
1977,Visione Restaurant and Living,Strada Nicolini Basso 34,Barbaresco,Piedmont,12050,Italy,€€€,"Contemporary, Piedmontese","At this restaurant, new, young and enthusiasti...","[Air conditioning, Car park]","[Amex, Maestrocard, Mastercard, Visa]",+39 328 134 0218,https://www.ristorantevisione.it
1978,Ristorante de LEN,Via Cesare Battisti 66,Cortina d'Ampezzo,Veneto,32043,Italy,€€,Regional Cuisine,Just a stone’s throw from the central and very...,[Wheelchair access],"[Amex, Dinersclub, Mastercard, Visa]",+39 0436 4246,https://hoteldelen.it
1979,AceroRosso,Via Ruvignan 1,Vodo di Cadore,Veneto,32040,Italy,€€,Regional Cuisine,This secluded mountain chalet immersed in verd...,"[Car park, Terrace, Wheelchair access]","[Amex, Mastercard, Visa]",+39 0435 489653,https://www.acerorossodolomiti.it
1980,Café Les Paillotes,piazza Le Laudi 2,Pescara,Abruzzo,65129,Italy,€€€,"Modern Cuisine, Seafood",This old acquaintance of the Michelin Guide no...,"[Air conditioning, Interesting wine list, Rest...","[Amex, Dinersclub, Mastercard, Visa]",+39 085 61809,https://www.lespaillotes.it/


In [ ]:
print(f"number of rows are: {data.shape[0]},number of columns are: {data.shape[1]}")

number of rows are: 1981,number of columns are: 13


---

# <h1> <b> 2.0.0 Search Engine and preprocessing the Text </b> <img src= "https://cdn-icons-png.flaticon.com/256/2857/2857376.png" width=100 style="vertical-align: middle"> </h1>

In [ ]:
# Mapping price_range column for more readability

price_range_labels = {'€': 'Economic',  '€€': 'Affordable','€€€': 'Expensive','€€€€': 'Luxury'}

data['priceRange'] = data['priceRange'].map(price_range_labels)

In [ ]:
data.head()

,restaurantName,address,city,region,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,O Me O Il Mare,Via Roma 45/47,Gragnano,Campania,80054,Italy,Luxury,"Italian Contemporary, Modern Cuisine","Known around the world as the town of pasta, G...","[Air conditioning, Interesting wine list, Whee...","[Amex, Dinersclub, Mastercard, Visa]",+39 081 620 0550,http://omeoilmare.com
1,Donevandro,via Garibaldi 2,Popoli,Abruzzo,65026,Italy,Affordable,"Contemporary, Seasonal Cuisine","Up until a few years ago, the owner-chef at th...",[Air conditioning],"[Mastercard, Visa]",+39 388 887 6858,http://www.donevandroristorante.it
2,Ape Vino e Cucina,Piazza Risorgimento 3,Alba,Piedmont,12051,Italy,Affordable,"Piedmontese, Contemporary",This attractive restaurant in the heart of Alb...,"[Air conditioning, Terrace, Wheelchair access]","[Amex, Dinersclub, Maestrocard, Mastercard, Visa]",+39 0173 363453,https://www.apewinebar.it/alba/
3,Da Bob Cook Fish,largo Parsano vecchio 16,Sorrento,Campania,80067,Italy,Affordable,Seafood,Working in partnership with the nearby fishmon...,"[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 081 1778 3873,https://www.dabobcookfish.com/
4,DA_MÓ,Via Bruno Buozzi 20,Matera,Basilicata,75100,Italy,Affordable,"Regional Cuisine, Contemporary","This new, restored restaurant in the upper par...","[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 0835 686548,https://www.damoristorante.it/


In [ ]:
data["description"].iloc[0]

'Known around the world as the town of pasta, Gragnano is still home to small-scale pasta-makers renowned for their top-quality pasta. To reinforce this fact, this restaurant is housed in a building (dating back to 1695) once used as pasta factory. Now boasting a modern, spacious dining room with a vaulted ceiling and open-view kitchen, the restaurant serves three tasting menus, all with strong links to the region but also featuring more creative touches. The region also takes pride of place on the wine list – it’s worth asking the talented and experienced sommelier for her recommendations.'

In [ ]:
# visualize all token of the first description of dataset
print(data["description"].iloc[0].split())

['Known', 'around', 'the', 'world', 'as', 'the', 'town', 'of', 'pasta,', 'Gragnano', 'is', 'still', 'home', 'to', 'small-scale', 'pasta-makers', 'renowned', 'for', 'their', 'top-quality', 'pasta.', 'To', 'reinforce', 'this', 'fact,', 'this', 'restaurant', 'is', 'housed', 'in', 'a', 'building', '(dating', 'back', 'to', '1695)', 'once', 'used', 'as', 'pasta', 'factory.', 'Now', 'boasting', 'a', 'modern,', 'spacious', 'dining', 'room', 'with', 'a', 'vaulted', 'ceiling', 'and', 'open-view', 'kitchen,', 'the', 'restaurant', 'serves', 'three', 'tasting', 'menus,', 'all', 'with', 'strong', 'links', 'to', 'the', 'region', 'but', 'also', 'featuring', 'more', 'creative', 'touches.', 'The', 'region', 'also', 'takes', 'pride', 'of', 'place', 'on', 'the', 'wine', 'list', '–', 'it’s', 'worth', 'asking', 'the', 'talented', 'and', 'experienced', 'sommelier', 'for', 'her', 'recommendations.']


### Before building the search engine, you must clean and prepare the text in each restaurant’s description. We will:

- #### 1. Remove stopwords ;
- #### 2. Remove punctuation ;
- #### 3. Apply stemming ;
- #### 4. Perform any other necessary cleaning to improve search accuracy.

### For the Preprocess operation of all description we use the `nltk` library

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize stop words and stemmer
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")


def text_cleaning(text):

    text=text.lower()

    # Remove special characters and symbols (non-alphanumeric characters)
    text = re.sub(r'[^\w\s]', '', text)

    # Remove extra spaces and numbers
    text = re.sub(r'\s+', '  ', text)
    text = re.sub(r'\d+', ' ', text)

    # Tokenize the text (split it into individual words)
    words = word_tokenize(text)

    # Apply filters, remove stopwords, exclude verbs, and apply stemmmer
    processed_words = [
        stemmer.stem(word)
        for word in words
        if word not in stop_words and nltk.pos_tag([word])[0][1] not in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]  ] # remove all stopwords, verbs from the description and apply the stemmer


    processed_text = ' '.join(processed_words).strip()

    return processed_text


# Apply the preprocessing function
tqdm.pandas(desc="Processing descriptions")  # Setup tqdm progress bar
data['description2'] = data['description'].progress_apply(text_cleaning)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Processing descriptions: 100%|██████████| 1981/1981 [00:34<00:00, 57.93it/s]


We have done preprocessing on the restaurant descriptions using the nltk library, following these steps:

*  Removal of stopwords: We imported English stopwords and kept only words that are not among them ;
*  Removal of punctuation ;
*  Application of stemming: We used the PorterStemmer to reduce words to their root ;
*  Other filters: We converted words to lowercase and accepted only alphabetical words.
---
At the end these operations were applied to the `description` column of the dataset, and the result was saved in a new column `description2` , ready for use in a search engine.

In [ ]:
# Set display options to show full text in each cell
pd.set_option('display.max_colwidth', None)

# Display the first few rows of 'description2'
print(data[['description2']].iloc[0])

description2    around world town pasta gragnano still home smallscal pastamak topqual pasta reinforc fact restaur build back pasta factori boast modern spacious room ceil openview kitchen restaur serv three menus strong link region also creativ touch region also pride place wine list worth experienc sommeli recommend
Name: 0, dtype: object


In [ ]:
#create a set of all noduplicated words in the variable 'description2'
vocabulary = set()  # Set to store unique words
for desc in data['description2']:
    # Tokenize the description and add unique words to the vocabulary
    vocabulary.update(desc.split())
print(f"number of words in vocabulary : {len(vocabulary)} words!")

#create a dictionary_word with all noduplicated words as keys and the associated term_id as values
dictionary_word={}
for word_id,word in enumerate(vocabulary,start=0):
    dictionary_word[word]=word_id

number of words in vocabulary : 7056 words!


In [ ]:
print(list(dictionary_word.items())[:50])

[('met', 0), ('trota', 1), ('pictur', 2), ('marci', 3), ('caldura', 4), ('vulcanello', 5), ('flora', 6), ('criteria', 7), ('abil', 8), ('vibe', 9), ('scalera', 10), ('cicchetteria', 11), ('chateaubriand', 12), ('cracker', 13), ('sarzana', 14), ('spell', 15), ('sourcherri', 16), ('pistachio', 17), ('mine', 18), ('villag', 19), ('main', 20), ('undiminish', 21), ('seem', 22), ('matilda', 23), ('visitor', 24), ('heard', 25), ('argaj', 26), ('sontuosi', 27), ('slice', 28), ('panfilio', 29), ('maggior', 30), ('neri', 31), ('pizzi', 32), ('substanti', 33), ('cusago', 34), ('accademia', 35), ('venosta', 36), ('quarta', 37), ('formazza', 38), ('thank', 39), ('santo', 40), ('taught', 41), ('snapper', 42), ('morsi', 43), ('grödnerhof', 44), ('wineri', 45), ('minimaliststyl', 46), ('monterozzi', 47), ('damiano', 48), ('cesar', 49)]


In [ ]:
#save dictionary_word in a csv file named "vocabulary.csv"
dictionary_word_df=pd.DataFrame(dictionary_word.items(),columns=['word','word_id'])
dictionary_word_df.to_csv('vocabulary.csv',index=False)

In [ ]:
#show first rows of vocabulary.csv
vocabulary=pd.read_csv('vocabulary.csv')
vocabulary.head()

,word,word_id
0,met,0
1,trota,1
2,pictur,2
3,marci,3
4,caldura,4


## 2.1 Conjunctive Query
### This first version of the search engine narrows the search to the description field of each restaurant. Only restaurants whose descriptions contain all the query words will be returned.
---
## 2.1.1 Create your Index!

* ### Vocabulary File: Create a file called vocabulary.csv that maps each word to a
unique integer (term_id);
* ### Inverted Index: Build a dictionary_word mapping each term_id to a list of document IDs where that term appears.
{
  "term_id_1": [document_1, document_2, document_4],
  "term_id_2": [document_1, document_3, document_5],
  ...
}

---
#### **Each document_i represents a unique restaurant.**

#### We've created a vocabulary of words from the description2 column. We assigned a unique term_id to each word and saved it to vocabulary.csv. This step is necessary for the construction of the inverted index.

In [ ]:
# Initialize the inverted index and dictionary_word for restaurant names
inverted_index = {}
dictionary_word_restaurant = {}

# Create dictionary_word for term to term_id mapping
# Assuming 'dictionary_word' will store words as keys and their corresponding term_ids as values
# Populate the dictionary_word for each unique restaurant name
for document_id, restaurant_name in enumerate(data['restaurantName'], start=0):
    dictionary_word_restaurant[restaurant_name] = document_id  # Map restaurant name to document id

# Initialize the inverted index from the 'description2' column
for i in range(len(data)):
    # Extract document_id from dictionary_word_restaurant using the restaurant name
    document_id = dictionary_word_restaurant[data['restaurantName'].iloc[i]]
    words = data['description2'].iloc[i].split()  # Split description2 into words

    # For each word in the description2
    for word in words:
        if word in dictionary_word:  # Ensure the word exists in the dictionary_word
            term_id = dictionary_word[word]  # Extract term_id from dictionary_word

            # Initialize the inverted index entry for this term_id if not present
            if term_id not in inverted_index:
                inverted_index[term_id] = []

            # Add the document_id to the inverted index for this term_id if it's not already present
            if document_id not in inverted_index[term_id]:
                inverted_index[term_id].append(document_id)

# Print the first word entries of the vocabulary (word, term_id pairs)
print("Vocabulary:", list(dictionary_word.items())[:1])

# Show the first word entries of the inverted index for inspection
print("Inverted Index:", list(inverted_index.items())[:1])

# Check if the number of unique term_ids in vocabulary matches the term_ids in the inverted index
if len(vocabulary) == len(inverted_index):
    print("The vocabulary and inverted index match.")
else:
    print("There is a mismatch between vocabulary and inverted index.")


Vocabulary: [('met', 0)]
Inverted Index: [(2467, [0, 10, 14, 25, 29, 32, 50, 65, 76, 97, 101, 109, 135, 159, 180, 189, 221, 293, 336, 353, 358, 375, 381, 435, 438, 444, 505, 516, 523, 567, 610, 615, 648, 663, 696, 704, 730, 732, 734, 754, 758, 760, 764, 809, 847, 853, 872, 889, 905, 919, 936, 937, 1012, 1043, 1046, 1088, 1109, 1175, 1189, 1223, 1247, 1255, 1279, 1280, 1288, 1315, 1345, 1352, 1353, 1380, 1425, 1435, 1454, 1573, 1578, 1591, 1665, 1703, 1708, 1757, 1781, 1786, 1848, 1909, 1923, 1929, 1931, 1934, 1941, 1949, 1953, 1980])]
The vocabulary and inverted index match.


**We've created :**
* A dictionary_word for restaurants, dictionary_word_restaurant, where each key is a unique restaurant name from the restaurantName column, and each value is a unique document_id ;
* A dictionary_word inverted_index, which contains term_id values as keys, each linked to a list of document_id values where the words appear.

Then we checked that inverted_index contains all the words from description2 by comparing the length of vocabulary with the number of keys in inverted_index. This confirms that all words are included in the inverted index

In [ ]:
# use pickle library to save inverted index in "inverted_index.pkl"

with open("inverted_index.pkl","wb") as f:
    pickle.dump(inverted_index,f)
# show inverted index
with open("inverted_index.pkl","rb") as f:
    loaded_inverted_index=pickle.load(f)

## 2.1.2 Execute the Query

#### **When the user inputs a query, for example, "modern seasonal cuisine", the search engine will:**

In [ ]:
def execute_query(query):

    query =  query.lower().split()

    # Create an empty list to store the word IDs corresponding to the query terms
    query_word_ids = []

    # Iterate over each word in the preprocessed query
    for word in query:
        # If the word exists in the dictionary (which maps words to IDs)
        if word in dictionary_word:
            # Append the corresponding word ID to the query_word_ids list
            query_word_ids.append(dictionary_word[word])

    # Check if we have any valid word IDs in the query
    if not query_word_ids:
        return "No matching words found in the dictionary."

    # Initialize the intersection list to store the document IDs that match the query
    intersection = []

    # If there are multiple query words, find the intersection of document IDs for all query terms
    if len(query_word_ids) >= 1:
        # Start with the set of document IDs corresponding to the first word in the query
        initial_set = set(inverted_index.get(query_word_ids[0], []))

        # For each remaining word ID in the query, find the intersection of document IDs
        for word_id in query_word_ids:
            current_set = set(inverted_index.get(word_id, []))
            intersection_set = initial_set & current_set  # Update the intersection

        # Convert the final intersection set to a list
        intersection = list(intersection_set)
    else:
        # If there is only one query word, use its document IDs directly
        intersection = list(set(inverted_index.get(query_word_ids[0], [])))

    # Check if the intersection is empty
    if not intersection:
        return "No documents match the query."

    # Initialize the list to store restaurant names that match the document IDs in the intersection
    restaurant_names = []

    # For each document ID in the intersection
    for doc_id in intersection:
        # Check if this document ID corresponds to any restaurant in the dictionary
        for name, rest_id in dictionary_word_restaurant.items():
            if rest_id == doc_id:
                # If the document ID matches, add the restaurant name to the list
                restaurant_names.append(name)

    # Check if any restaurants were found
    if not restaurant_names:
        return "No matching restaurants found."

    # Filter the original data to only include the restaurants in the restaurant_names list
    results = data[data["restaurantName"].isin(restaurant_names)]

    # Select only the relevant columns for the final results
    results = results[['restaurantName', 'address', 'description', 'website']]

    # Return the search results as a DataFrame
    return results




* Processed query words: the function cleans and converts all query words to ensure compatibility with the inverted index
*  Conjunctive Query: an intersection of all document_ids for the query words is performed, so that only restaurants containing all the words in the query are returned
*  Output: the result is a DataFrame with the columns {restaurantName, address, description, website}

We have implemented the conjunctive search engine, and the output provides results for the query 'modern seasonal cuisine'




In [ ]:
query = input("Please enter a short description of the type of restaurant you're looking for:\n"
              "For example: 'modern seasonal cuisine with a terrace' or 'romantic Italian restaurant'.\n\n"
              "Your description: ")

results = execute_query(query)

# Show the first five results
results[:5]

Please enter a short description of the type of restaurant you're looking for:
For example: 'modern seasonal cuisine with a terrace' or 'romantic Italian restaurant'.

Your description:  modern seasonal cuisine


,restaurantName,address,description,website
0,O Me O Il Mare,Via Roma 45/47,"Known around the world as the town of pasta, Gragnano is still home to small-scale pasta-makers renowned for their top-quality pasta. To reinforce this fact, this restaurant is housed in a building (dating back to 1695) once used as pasta factory. Now boasting a modern, spacious dining room with a vaulted ceiling and open-view kitchen, the restaurant serves three tasting menus, all with strong links to the region but also featuring more creative touches. The region also takes pride of place on the wine list – it’s worth asking the talented and experienced sommelier for her recommendations.",http://omeoilmare.com
1,Donevandro,via Garibaldi 2,"Up until a few years ago, the owner-chef at this restaurant was working as a painter – a fact that is evident from the artistic touch in his cuisine. His recipes are modern and personalised, with careful attention naturally paid to harmonious presentation, while the flavour of his dishes is brought out by ingredients that are skilfully chosen from the Abruzzo inland area. In 2024, the restaurant moved to new, centrally located premises which have an intimate feel and are elegant and minimalist in style.",http://www.donevandroristorante.it
8,La Buca,corso Garibaldi 45,"Choose one of the tables on the outdoor summer terrace (right over the water!) at this restaurant in order to best appreciate its location overlooking the picturesque canal port and its period houses. The indoor dining room is also attractive, with its open-view kitchen and a modern and elegant feel, while the menu focuses almost exclusively on fish, most of which is sourced from local fishermen. Enjoy a wide selection of raw antipasti to start (some simple in their preparation, while others are more elaborate), then choose from an array of beautifully presented, modern dishes, perhaps accompanied by one of the many champagnes included on the wine list.",https://www.labucaristorante.com/
11,Il Ristorante Alain Ducasse Napoli,Via Cristoforo Colombo 45,"Alain Ducasse, one of the great names in contemporary fine dining, has arrived in Naples, opening this restaurant in the former premises of the prestigious Ristorante Il Comandante. Situated on the 9th floor of the Romeo hotel overlooking the port, the restaurant boasts fine views of Vesuvius and the Bay of Naples, especially at sunset when the colours are truly spectacular. Meanwhile, the modern dining room, decorated completely in black, is equally as stunning as its surroundings. Alessandro Lucassino, who was born in 1991 and has years of experience working with his mentor, adds a personal flavour to local recipes: by using short cooking times, he preserves the nutritious qualities and flavours of local fish and vegetables while remaining faithful to Ducasse’s “cuisine de la naturalité” philosophy.",https://theromeocollection.com/en/romeo-napoli/restaurants-bars/il-ristorante-alain-ducasse/
14,Etra,piazza De Ferrari 4,"Etra is an anagram of the Italian word “arte” – an apt name for this restaurant, which is housed in Palazzo Doria De Fornari on the beautiful and famous Piazza Ferrari. This palazzo is one of Genova’s magnificent “Rolli” palazzi (noble palaces once used to house famous visitors to the city). Despite the setting, the dining room is modern in style and adorned with elegant works of contemporary art. Here, chef Davide Cannavino consolidates his reputation with a concise menu of creative dishes: around a dozen meat and fish options inspired by the beauty that surrounds him and showcased on two tasting menus from which individual dishes can be chosen à la carte style.",https://www.etra.art/


## 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity
### For the second search engine, given a query, retrieve the top-k restaurants ranked by relevance to the query.
---
## 2.2.1 Inverted Index with TF-IDF Scores
- ### tfIdf Scores: Calculate TF-IDF scores for each term in each restaurant’s description.
- ### Updated Inverted Index: Build a new inverted index where each entry is a term, and the value is a list of tuples containing document IDs and TF-IDF scores.

In [ ]:
def calculate_tf(data, dictionary_word_restaurant):
    """Calculate term frequency (TF) for each document and track document appearances for each word."""
    tf = {}
    dict_word_doc_id = {}

    # Iterate over each document in the dataset
    for idx, row in data.iterrows():
        # Get the document ID from the 'restaurantName'
        document_id = dictionary_word_restaurant.get(row['restaurantName'])

        # Getting the list of words from the preprocessed description
        words = row['description2'].split()

        # Initialize a dictionary for TF scores for this document
        tf[document_id] = {}

        for word in words:
            # Increment the word count in the current document's dictionary
            tf[document_id][word] = tf[document_id].get(word, 0) + 1

            # Track the document ID for the current word
            if word not in dict_word_doc_id:
                dict_word_doc_id[word] = set()
            dict_word_doc_id[word].add(document_id)

        # Normalize TF values by dividing by the total number of words in the document
        num_words = len(words)
        for word in tf[document_id]:
            tf[document_id][word] /= (math.log(num_words)+1)

    return tf, dict_word_doc_id


---
* ### Calculated TF-IDF scores: the calculate_tf_idf function computed the TF-IDF scores for each word in the descriptions.
* ### Created updated inverted index: this inverted index uses term_id as keys and stores tuples of (document_id, tfidf_score) as values.
---
This updated inverted index is now ready to be used in the Ranked Search Engine with TF-IDF and Cosine Similarity


In [ ]:
def calculate_tf_idf(data, dictionary_word_restaurant):
    """Calculates TF-IDF for each document and creates the inverted index."""
    # Calculate TF and build the document index for each word
    tf, dict_word_doc_id = calculate_tf(data, dictionary_word_restaurant)
    N = len(data)  # Total number of documents

    tf_idf = {}  # Dictionary to store TF-IDF scores

    # Calculate IDF and TF-IDF
    for document_id, word_tf in tf.items():
        tf_idf[document_id] = {}
        for word, tf_score in word_tf.items():
            # Calculate IDF
            idf = math.log(N / len(dict_word_doc_id[word]))
            # Calculate TF-IDF as tf * idf
            tf_idf[document_id][word] = tf_score * idf

    return tf_idf, dict_word_doc_id


In [ ]:
def create_inverted_index(tf_idf, dictionary_word):
    """Create an inverted index mapping terms to documents and their TF-IDF scores."""
    inverted_index = {}

    for document_id, word_tfidf in tf_idf.items():
        for word, tfidf_score in word_tfidf.items():
            # Get the ID for the term from the dictionary
            term_id = dictionary_word.get(word)

            if term_id is None:
                raise KeyError(f"Term '{word}' not found in dictionary_word.")

            # Add the document and the TF-IDF score to the term's list in the inverted index
            if term_id not in inverted_index:
                inverted_index[term_id] = []
            inverted_index[term_id].append((document_id, tfidf_score))

    return inverted_index

# Execute the TF-IDF calculation and create the inverted index
tf_idf, dict_word_doc_id = calculate_tf_idf(data, dictionary_word_restaurant)
inverted_index = create_inverted_index(tf_idf, dictionary_word)

## 2.2.2 Execute the Ranked Query
---
### Ranking Restaurants using Cosine Similarity

The process involves ranking restaurants based on a search query through the following steps:

#### 1. **Process the Query Terms**: The query terms are extracted and prepared.
#### 2. **Calculate TF-IDF Vectors**: Compute the TF-IDF vectors for both the query and each restaurant document.
#### 3. **Compute Cosine Similarity**: Compare the cosine similarity between the query and each restaurant’s TF-IDF vector.
#### 4. **Return Top-k Results**: Return the top-k restaurants or fewer if there are less than k matches with non-zero similarity.
#### 5. **Result Details**: Each result should include:
   - `Restaurant Name`
   - `Address`
   - `Description`
   - `Website`
   - `Similarity Score (between 0 and 1)`


### Now create restaurant_info_dict as a base to access restaurant details, like restaurant_name, address, description , website etc... This restaurant_info_dict will be used in the next steps

In [ ]:
restaurant_info_dict={}

for i in range(len(data)):
    document_id=dictionary_word_restaurant[data['restaurantName'].iloc[i]]  #used dictionary_word_restaurant created previously to obtain `document_id`

    if document_id not in restaurant_info_dict:
    # Only select specific columns
        restaurant_info_dict[document_id]={
            "restaurantName": data['restaurantName'].iloc[i],
            "city": data['city'].iloc[i],
            "region": data['region'].iloc[i],
            "address": data['address'].iloc[i],
            "description": data['description'].iloc[i],
            "website": data['website'].iloc[i],
            "cuisineType": data['cuisineType'].iloc[i],
            "priceRange": data['priceRange'].iloc[i],
            "facilitiesServices": data['facilitiesServices'].iloc[i]
        }

In [ ]:
# Calculate TF-IDF for the query

def calculate_query_tfidf(preprocess_query, dict_word_doc_id, N):
    tf_query = {}
    # Calculate Term Frequency (TF) for each word in the query
    for word in preprocess_query:
        tf_query[word] = tf_query.get(word, 0) + 1

    number_word = len(preprocess_query)

    # Calculate TF-IDF scores for the query
    tfidf_query = {}
    for word, tf in tf_query.items():

        # Only include words that exist in our document collection
        if word in dict_word_doc_id:
            normalized_tf = tf / number_word  # Normalize TF by query length
            idf = math.log(N / len(dict_word_doc_id[word]))  # Calculate IDF
            tfidf_query[word] = normalized_tf * idf

    return tfidf_query

# Cosine Similarity calculation
def cosine_similarity(vector1, vector2):
    # Calculate dot product
    dot_product = sum(vector1[word] * vector2.get(word, 0) for word in vector1)

    # Calculate Euclidean norms
    norm1 = math.sqrt(sum(value ** 2 for value in vector1.values()))
    norm2 = math.sqrt(sum(value ** 2 for value in vector2.values()))

    # Ensure norms are non-zero to prevent division by zero
    return dot_product / (norm1 * norm2) if norm1 and norm2 else 0

# Ranking function to get top k results
def ranking_function(query, tf_idf_data, k, dict_word_doc_id, N, restaurant_info_dict):

    # Calculate TF-IDF for the query
    tfidf_query = calculate_query_tfidf(query, dict_word_doc_id, N)

    # Calculate cosine similarity for each document
    cos_sim = []
    for document_id, tfidf_vector in tf_idf_data.items():
        score = cosine_similarity(tfidf_query, tfidf_vector)
        if score > 0:  # Only consider documents with a positive similarity score
            cos_sim.append((document_id, score))

    # Sort results by similarity score in descending order and take top k
    cos_sim_sorted = sorted(cos_sim, key=lambda x: x[1], reverse=True)[:k]

    # Create the final results with details
    final_results = []
    for document_id, score in cos_sim_sorted:
        restaurant = restaurant_info_dict[document_id]
        restaurant["Similarity score"]=round(score, 4)
        final_results.append(restaurant)

    return pd.DataFrame(final_results)


In [ ]:
# Prepare the query
query = input("Please enter a short description of the type of restaurant you're looking for, including the type of cuisine you prefer.\n"
    "For example: 'modern seasonal cuisine with a terrace', 'romantic Italian restaurant', or 'traditional Japanese sushi bar'.\n\n"
    "Your description: ")
preprocess_query=text_cleaning(query)

k=50 # We choose to consider only first 50 restaurants with higher similarity score

# Execute Ranking function
ranking_results=ranking_function(preprocess_query,tf_idf,k,dict_word_doc_id,len(data),restaurant_info_dict)

# Show the result
ranking_results.head()

Please enter a short description of the type of restaurant you're looking for, including the type of cuisine you prefer.
For example: 'modern seasonal cuisine with a terrace', 'romantic Italian restaurant', or 'traditional Japanese sushi bar'.

Your description:  modern seasonal cuisine


,restaurantName,city,region,address,description,website,cuisineType,priceRange,facilitiesServices,Similarity score
0,Matteo Ristorante,Biella,Piedmont,piazza Duomo 6,"The soft-toned decor is modern and intimate, while the cuisine also has a contemporary feel. Dishes include meat and fish options, plus numerous types of risottos, many of which feature distinctly modern ingredients. Coffee service and aperitifs in the near Laboratory, at n 10.",https://matteocaffeecucina.it/,Modern Cuisine,Affordable,"[Air conditioning, Terrace, Wheelchair access]",0.4056
1,Il Cavallo Scosso,Asti,Piedmont,via al Duca 23/d,"A young and modern restaurant situated in a residential area about 2km from the centre, where the owner-chef offers three tasting menus:Territorio e Tradizione,Il Cavallo Scossoand the more creativeFuori Gara.",https://www.ilcavalloscosso.it/,"Contemporary, Seafood",Affordable,"[Air conditioning, Car park]",0.0744
2,Casa Perbellini 12 Apostoli,Verona,Veneto,vicolo Corticella San Marco 3,"Giancarlo Perbellini returns to his origins in this historic restaurant in his native city, where every chef from Verona (and possibly beyond) would like to work. The updated decor gives the restaurant even more appeal, as does the culinary style and philosophy showcased on three tasting menus: “Io e Silva”, which includes decidedly imaginative dishes (such as cooked and raw shellfish with a dash of soya and peppers) and is dedicated to his wife; “Io e Giorgio”, dedicated to his mentor and former restaurant owner, which features more classic recipes; and, last but not least, the completely vegetarian “L'Essenza”. Special mention must be made of the wine list, which includes an extensive selection of French labels, of which both the sommelier and Giancarlo are huge fans. As a final attraction, make sure you find time to visit the Roman ruins in the basement. We also highly recommend booking the chef's table where you can dine as a couple yet at the same time enjoy the company of the skilled chefs whom you’ll observe working together like the well-practised members of an orchestra – an extraordinary sight!",http://www.casaperbellini.com,"Creative, Contemporary",Luxury,"[Air conditioning, Counter dining, Interesting wine list, Restaurant offering vegetarian menus, Wheelchair access]",0.0727
3,Trattoria Pennestri,Rome,Lazio,via Giovanni Da Empoli 5,"An authentic neighbourhood trattoria with just one difference – it is so good that its fame has travelled far beyond the streets of Ostiense in which it is situated and so it is often crowded (as a result, booking is highly recommended). The atmosphere is simple, informal and attractive, while the cuisine focuses on Roman classics such as pasta with carbonara, cacio e pepe, amatriciana and gricia sauces, alongside a few more creative dishes.",https://trattoriapennestri.it/,"Cuisine from Lazio, Seasonal Cuisine",Economic,"[Air conditioning, Terrace, Wheelchair access]",0.0686
4,I Due Buoi,Olivola,Piedmont,via Vittorio Veneto 23,"This restaurant offers three tasting menus: Stile Libero, Tradizione e Territorio, and Quinto Quarto , the latter featuring various types of offal which the chef transforms into creative dishes, plus an à la carte that includes local specialities and a good choice of bread. Top-quality wines with a focus on Asti and its surrounding area, as well as on the Langhe region.",https://www.iduebuoi.it,"Piedmontese, Modern Cuisine",Affordable,[Car park],0.0676


---

# <h1> <b> 3.0.0 Define a New Score! </b> <img src="https://raw.githubusercontent.com/Heibattttt/Michelin-restaurant-in-Italy-web-scraping/main/Images/img%20point%203.PNG" alt="Restaurant Score Comparison" width=150 style="vertical-align: middle"> </h1>

## New Scoring function:
---
### Define a scoring function that takes into account various attributes:

- #### `Description Match`: Give weight based on the query similarity to the description (using TF-IDF scores);
- #### `Cuisine Match`: Increase the score for matching cuisine types;
- #### `Facilities and Services`: Give more points for matching facilities/services (e.g., “Terrace,” “Air conditioning”);
- #### `Price Range`: Higher scores could be given to more affordable options based on the user’s choice.

In [ ]:
# Scoring function that incorporates description, cuisine, services and price range

def custom_scoring(restaurant,prefer_servicies,preprocess_query):
    cuisine_score=0
    servicies_score=0
    # 1. Description Match (Cosine Similarity)
    description_score = restaurant["Similarity score"]

    # 2. Cuisine Match (Boost score for matching cuisine type)
    cuisine_types = restaurant['cuisineType'].lower().split(',')
    cuisine_score += sum(1 for cuisine in cuisine_types if cuisine in query) # Add a constant boost for matching cuisine type

    # 3. Facilities and Services Match (Add points for matching facilities/services)
    for facility in restaurant['facilitiesServices']:
        if facility.lower() in prefer_servicies:
            servicies_score += 0.5
        else:
            servicies_score += 0.2

    # 4. Price Range (Match based on user’s budget, if provided)
    if restaurant['priceRange'] == "Economic":
        price_score = 2
    elif restaurant['priceRange'] == "Affordable":
        price_score = 1
    else:
        price_score = 0

    return description_score*0.4 + cuisine_score*0.3 + servicies_score*0.2 + price_score*0.1

# Main Ranking Function that uses Heap for Top-k

def ranking_function_with_custom_score(ranking_df, prefer_servicies,preprocess_query):

    heap = []   # Initialize an empty heap

    # Loop through all restaurants
    for index in range(ranking_df.shape[0]):

        # Copy restaurant data in i position
        restaurant = ranking_df.iloc[index].copy()

        # Calculate custom score
        custom_score = custom_scoring(restaurant, prefer_servicies,preprocess_query)

        # Add custom score to restaurant
        restaurant["Similarity score"] = custom_score

        # Push (score, restaurant) tuple into the heap
        heapq.heappush(heap, (custom_score, restaurant))

    # Get top-k (50) restaurants based on custom score
    top_k_results = heapq.nlargest(50, heap, key=lambda x: x[0])  # Sort by custom score

    # Convert to DataFrame, extracting only restaurants
    top_k_results_df = pd.DataFrame([item[1] for item in top_k_results])

    # Return the top-k restaurants
    return top_k_results_df

In [ ]:
# Example usage and results:

servicies = input("Enter the services offered that you want :\n" "For example: Terrace, Air conditioning.\n: ")
servicies_list = [service.lower() for service in servicies.split(',')]

custom_results = ranking_function_with_custom_score(ranking_results,servicies_list,preprocess_query)
custom_results.head()

Enter the services offered that you want :
For example: Terrace, Air conditioning.
:  Terrace, Air conditioning.


,restaurantName,city,region,address,description,website,cuisineType,priceRange,facilitiesServices,Similarity score
3,Trattoria Pennestri,Rome,Lazio,via Giovanni Da Empoli 5,"An authentic neighbourhood trattoria with just one difference – it is so good that its fame has travelled far beyond the streets of Ostiense in which it is situated and so it is often crowded (as a result, booking is highly recommended). The atmosphere is simple, informal and attractive, while the cuisine focuses on Roman classics such as pasta with carbonara, cacio e pepe, amatriciana and gricia sauces, alongside a few more creative dishes.",https://trattoriapennestri.it/,"Cuisine from Lazio, Seasonal Cuisine",Economic,"[Air conditioning, Terrace, Wheelchair access]",0.70744
35,Molin Vecio,Caldogno,Veneto,via Giaroni 116,"In a 16C working mill, rooms full of atmosphere (one with a fireplace) and service in summer on the lakeside; typical Vicenza cuisine with vegetarian dishes.",http://www.molinvecio.it,"Venetian, Seasonal Cuisine",Affordable,"[Car park, Garden or park, Terrace, Wheelchair access]",0.62748
17,Trattoria della Fortuna,Monterotondo,Lazio,Via Salaria 57,"This interior of this trattoria occupying an anonymous rural building at a road junction just outside the village comes as something of a surprise. Inside, the dining room has a warm ambience, with a friendly welcome offered by the female staff. The dishes here are authentic and full of flavour, with a focus on regional Italian traditions and top-quality local ingredients – all the pasta and desserts are homemade. We particularly recommend two of the specialities, namely the “Girella e Cartellata” (made using 36 egg yolks) stuffed with ricotta and goat’s cheese and served with a “ciauscolo” (a type of sausage typical of the Marche and Umbria) ragu and a red wine reduction. A small, simple pergola-shaded outdoor space completes the picture.",http://www.trattoriadellafortuna.it,"Regional Cuisine, Seasonal Cuisine",Affordable,"[Air conditioning, Terrace, Wheelchair access]",0.60080
0,Matteo Ristorante,Biella,Piedmont,piazza Duomo 6,"The soft-toned decor is modern and intimate, while the cuisine also has a contemporary feel. Dishes include meat and fish options, plus numerous types of risottos, many of which feature distinctly modern ingredients. Coffee service and aperitifs in the near Laboratory, at n 10.",https://matteocaffeecucina.it/,Modern Cuisine,Affordable,"[Air conditioning, Terrace, Wheelchair access]",0.44224
12,Hosteria Grappolo d'Oro,Rome,Lazio,piazza della Cancelleria 80,"Since the turn of the century, this long-established restaurant near Piazza Navona and Campo de' Fiori has been run by five local business partners, one of whom is the chef. Top-quality, classic and generous Roman cuisine (cacio e pepe pasta, roast lamb, salted cod etc) is to the fore here, served in an attractive, rustic-style dining room. The background music – a nostalgic 1970s mix of rock and blues – also deserves a mention.",https://hosteriagrappolodoro.it/,"Roman, Traditional Cuisine",Economic,"[Air conditioning, Terrace, Wheelchair access]",0.40316


## <h1> <b> Cosine Similarity </b> <img src="https://raw.githubusercontent.com/Heibattttt/Michelin-restaurant-in-Italy-web-scraping/main/Images/vs-removebg-preview.png" width=80 style="vertical-align: middle"> <b> Custom Score </b> </h1>


The personalized scoring function provides superior results by going beyond simple keyword matching based solely on cosine similarity values based on the **TF-IDF** vectors of the query and each document. This new function takes into account more factors related to user preferences, providing more relevant matches while improving user satisfaction.

---

## Comparison of two methods

The search results differ significantly between the methods:

- Using only **cosine similarity**, *"Matteo Ristorante"* scored **0.4**, while the remaining **k restaurants** scored between **0** and **0.1**, indicating that considering only this parameter is not a good criterion.

- In contrast, the **personalized scoring method**, which considers more factors related to user preferences, provided higher matching scores in all k restaurants, demonstrating that considering more related factors can help in building a better search engine.



---

# <h1> <b> 4.0.0 Visualizing the Most Relevant Restaurants </b> <img src="https://styles.redditmedia.com/t5_1k462c/styles/profileIcon_snoo-nftv2_bmZ0X2VpcDE1NToxMzdfYzhkM2EzYTgzYmRlNWRhZDA2ZDQzNjY5NGUzZTIyYWMzZTY0ZDU3N18yMzI4OTY_rare_5db9077f-09ef-493d-acb3-f60cc8055e24-headshot.png?width=256&height=256&frame=1&auto=webp&crop=256:256,smart&s=98431d8ebd76460a39af9aed01ddb9f682700b71" width=200 style="vertical-align: middle"> </h1>

### Maps can provide users with an easy way to see where restaurants are located. This is especially useful for understanding which regions in Italy have the most options. The next step is to graph the restaurants with the best scores in the 3.0.0 point.

In [ ]:
#!!!don't run this code!!! import df dataset in the following step
!pip install opencage
from opencage.geocoder import OpenCageGeocode
import pandas as pd
# Insert my API key of OpenCage
key='4101055b9fa84cbdbe8b71af558c30ec'
geocoder=OpenCageGeocode(key)
#obtain a list of unique cities
list_cities=data['city'].unique()
coordinates=[]
for i in list_cities:
    location=geocoder.geocode(i)
    if location:
        coordinates.append({"city": i, "latitude": location[0]['geometry']['lat'] , "longitude": location[0]['geometry']['lng']})
df=pd.DataFrame(coordinates)
df


  <div id="df-8c784098-c956-4c32-99f5-d7fd93b712a7" class="colab-df-container">
    <div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>city</th>
      <th>latitude</th>
      <th>longitude</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Gragnano</td>
      <td>40.690000</td>
      <td>14.521065</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Popoli</td>
      <td>42.171311</td>
      <td>13.832817</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Alba</td>
      <td>46.015936</td>
      <td>23.546850</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Sorrento</td>
      <td>40.624906</td>
      <td>14.374836</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Matera</td>
      <td>40.447642</td>
      <td>16.473574</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <th>1163</th>
      <td>Capoterra</td>
      <td>39.175313</td>
      <td>8.971950</td>
    </tr>
    <tr>
      <th>1164</th>
      <td>Pisa</td>
      <td>43.471472</td>
      <td>10.679791</td>
    </tr>
    <tr>
      <th>1165</th>
      <td>Cardano</td>
      <td>46.493717</td>
      <td>11.393647</td>
    </tr>
    <tr>
      <th>1166</th>
      <td>Badalucco</td>
      <td>43.915557</td>
      <td>7.847295</td>
    </tr>
    <tr>
      <th>1167</th>
      <td>loc. Tre Stelle</td>
      <td>45.464758</td>
      <td>8.212023</td>
    </tr>
  </tbody>
</table>
<p>1168 rows × 3 columns</p>
</div>
    <div class="colab-df-buttons">

  <div class="colab-df-container">
    <button class="colab-df-convert" onclick="convertToInteractive('df-8c784098-c956-4c32-99f5-d7fd93b712a7')"
            title="Convert this dataframe to an interactive table."
            style="display:none;">

  <svg xmlns="http://www.w3.org/2000/svg" height="24px" viewBox="0 -960 960 960">
    <path d="M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z"/>
  </svg>
    </button>






In [ ]:
# Save the dataset 'df' because OpenCage allows only 2500 requests per day for free accounts
df.to_csv("coordinates.csv",index=False)

In [ ]:
coordinates = pd.read_csv("coordinates.csv")
coordinates.shape

(1168, 3)

The following operations is to ensure that city names match exactly in both DataFrames `custom_results` & `coordinates` , which is crucial for a successful merge operation.

In [ ]:
# Clean city names
custom_results['city'] = custom_results['city'].str.strip().str.capitalize()

# Clean city names in coordinates
coordinates['city'] = coordinates['city'].str.strip().str.capitalize()

### 1. In the following map created using the `Folium` library, the first 100 restaurants of the custom score ranking will be shown.
---
### 2. To improve the visualization, we will be used a color scale based on the price range of the k restaurants.

In [ ]:
# Merge the custom_results DataFrame with coordinates based on the city column

union = pd.merge(custom_results, coordinates, on="city", how="left")
# Drop any rows with missing values to ensure that only complete data is used
union = union.dropna()
union.head(5)

,restaurantName,city,region,address,description,website,cuisineType,priceRange,facilitiesServices,Similarity score,latitude,longitude
0,Trattoria Pennestri,Rome,Lazio,via Giovanni Da Empoli 5,"An authentic neighbourhood trattoria with just one difference – it is so good that its fame has travelled far beyond the streets of Ostiense in which it is situated and so it is often crowded (as a result, booking is highly recommended). The atmosphere is simple, informal and attractive, while the cuisine focuses on Roman classics such as pasta with carbonara, cacio e pepe, amatriciana and gricia sauces, alongside a few more creative dishes.",https://trattoriapennestri.it/,"Cuisine from Lazio, Seasonal Cuisine",Economic,"[Air conditioning, Terrace, Wheelchair access]",0.70744,41.893320,12.482932
1,Molin Vecio,Caldogno,Veneto,via Giaroni 116,"In a 16C working mill, rooms full of atmosphere (one with a fireplace) and service in summer on the lakeside; typical Vicenza cuisine with vegetarian dishes.",http://www.molinvecio.it,"Venetian, Seasonal Cuisine",Affordable,"[Car park, Garden or park, Terrace, Wheelchair access]",0.62748,45.611836,11.507580
2,Trattoria della Fortuna,Monterotondo,Lazio,Via Salaria 57,"This interior of this trattoria occupying an anonymous rural building at a road junction just outside the village comes as something of a surprise. Inside, the dining room has a warm ambience, with a friendly welcome offered by the female staff. The dishes here are authentic and full of flavour, with a focus on regional Italian traditions and top-quality local ingredients – all the pasta and desserts are homemade. We particularly recommend two of the specialities, namely the “Girella e Cartellata” (made using 36 egg yolks) stuffed with ricotta and goat’s cheese and served with a “ciauscolo” (a type of sausage typical of the Marche and Umbria) ragu and a red wine reduction. A small, simple pergola-shaded outdoor space completes the picture.",http://www.trattoriadellafortuna.it,"Regional Cuisine, Seasonal Cuisine",Affordable,"[Air conditioning, Terrace, Wheelchair access]",0.60080,42.051799,12.618397
3,Matteo Ristorante,Biella,Piedmont,piazza Duomo 6,"The soft-toned decor is modern and intimate, while the cuisine also has a contemporary feel. Dishes include meat and fish options, plus numerous types of risottos, many of which feature distinctly modern ingredients. Coffee service and aperitifs in the near Laboratory, at n 10.",https://matteocaffeecucina.it/,Modern Cuisine,Affordable,"[Air conditioning, Terrace, Wheelchair access]",0.44224,45.566954,8.086912
4,Hosteria Grappolo d'Oro,Rome,Lazio,piazza della Cancelleria 80,"Since the turn of the century, this long-established restaurant near Piazza Navona and Campo de' Fiori has been run by five local business partners, one of whom is the chef. Top-quality, classic and generous Roman cuisine (cacio e pepe pasta, roast lamb, salted cod etc) is to the fore here, served in an attractive, rustic-style dining room. The background music – a nostalgic 1970s mix of rock and blues – also deserves a mention.",https://hosteriagrappolodoro.it/,"Roman, Traditional Cuisine",Economic,"[Air conditioning, Terrace, Wheelchair access]",0.40316,41.893320,12.482932


In [ ]:
import folium  # Import the Folium library for creating maps
from IPython.display import display  # Import display function for rendering the map inline in Jupyter Notebook

# Create map centered to Italy
map = folium.Map(location=[41.8719, 12.5674], zoom_start=6)

# Define the price_color function
def price_color(price):
    if price == "Economic":
        return "green"
    elif price == "Affordable":
        return "blue"
    elif price == "Expensive":
        return "orange"
    elif price == "Luxury":
        return "red"

# Add markers for each restaurant
for i in range(len(union)):
    restaurant = union["restaurantName"].iloc[i]
    city = union["city"].iloc[i]
    address = union["address"].iloc[i]
    website= union["website"].iloc[i]
    price_range = union["priceRange"].iloc[i]
    custom_score = union["Similarity score"].iloc[i]
    latitude = union["latitude"].iloc[i]
    longitude = union["longitude"].iloc[i]
    region=  union["region"].iloc[i]

    website_link = f'<a href="{website}" target="_blank">{website}</a>'

    # Create popup with information on the map
    popup_text = f"<b>{restaurant}</b> <br>Address:{address} <br>Website:{website_link} <br> City:{city} <br> Region:{region} <br>Custom score: {custom_score:.4f}"
    folium.Marker(
        location=[latitude, longitude],
        popup=folium.Popup(popup_text, max_width=250),
        icon=folium.Icon(color=price_color(price_range))
    ).add_to(map)

# Create legend with price range
legend = '''
<div style="
    position:fixed;
    top:10px;
    left:10px;
    width:300x;
    border:1px solid grey;
    background-color:white;
    padding:15px;
    z-index:9999;
    border-radius: 8px;
">
    <b style="font-size: 18px;">Price Range Legend</b><br>
    <div style="display: flex; align-items: center;">
        <span style="color:green; font-size: 24px;">●</span>
        <span style="margin-left: 5px; font-size: 16px;">€ Economic</span>
    </div>
    <div style="display: flex; align-items: center;">
        <span style="color:blue; font-size: 24px;">●</span>
        <span style="margin-left: 5px; font-size: 16px;">€€ Affordable</span>
    </div>
    <div style="display: flex; align-items: center;">
        <span style="color:orange; font-size: 24px;">●</span>
        <span style="margin-left: 5px; font-size: 16px;">€€€ Expensive</span>
    </div>
    <div style="display: flex; align-items: center;">
        <span style="color:red; font-size: 24px;">●</span>
        <span style="margin-left: 5px; font-size: 16px;">€€€€ Luxury</span>
    </div>
</div>
'''

# Add legend to map
map.get_root().html.add_child(folium.Element(legend))

# Display the map inline
display(map)

#Save the mapp in html file
map.save("top_k_restaurants_map.html")

# <h1> <b> BONUS: Advanced Search Engine </b><img src="https://github.com/Heibattttt/Michelin-restaurant-in-Italy-web-scraping/blob/main/Images/bonus%20image.PNG?raw=true" width=200 style="vertical-align: middle"> </h1>

# Advanced Search Engine Function

This function of `search_engine_filters` module creates an interactive search engine for filtering restaurant data based on various criteria. The user can filter restaurants using a combination of region, city, price range, cuisines, accepted credit cards, and available services. It uses widgets to build a user-friendly interface.

---
## Features:
1. **Region & City filters**:
   - Dropdown menus allow the user to select a region and city;
   - Cities are dynamically updated based on the selected region.

2. **Price range filters**:
   - The user can select a minimum and maximum price range using predefined categories: `Economic`, `Affordable`, `Expensive` and `Luxury`.

3. **Cuisine filters**:
   - Available only if the `cuisineType` column exists in the dataset;
   - Checkboxes are displayed for each cuisine available in the selected city, allowing the user to select one or more options.

4. **Accepted credit cards & Services filters**:
   - The user can filter restaurants based on the accepted credit cards and available services (e.g., parking, outdoor seating, etc.).

5. **Dynamic data updates**:
   - The list of cities and cuisines updates based on previous selections;
   - The interface only displays relevant options to the user.

6. **Results display**:
   - After applying filters, the function displays a table of restaurants matching the selected criteria;
   - Each restaurant's website (if available) is shown as a clickable link.

7. **Reset option**:
   - A "reset" button clears all selections and restores the default state of the filters.
---
## How it works:
- **Step 1**: User selects region, city, price range, cuisines, cards, and services;
- **Step 2**: Filters are applied to the dataset;
- **Step 3**: The filtered results are displayed with restaurant details.

This interactive engine helps users quickly find restaurants based on specific preferences, improving the search experience.


In [ ]:
import search_engine_filters as sef
help(sef) # Information about search search_engine_filters module

Help on module search_engine_filters:

NAME
    search_engine_filters

FUNCTIONS
    Advanced_Search_Engine(dataframe)
        Creates an interactive search interface for filtering restaurant data.
        
        Args:
            dataframe (pandas.DataFrame): DataFrame containing restaurant information
            Required columns: restaurantName, region, city, priceRange, cuisineType, creditCards, facilitiesServices, website, address
            Optional columns: phonenumber, description
        
        Output:
               The function returns as output a table in HTML format with the restaurants that match the criteria selected in input by the user. 
               The table will contain the name of the restaurant, the city, the address, website, the type of cuisine, accepted cards and available services.

FILE
    c:\users\flavi\search_engine_filters.py




In [ ]:
data=pd.read_csv("all_restaurants_data.csv")
price_range_labels = {'€': 'Economic',  '€€': 'Affordable','€€€': 'Expensive','€€€€': 'Luxury'}
data['priceRange'] = data['priceRange'].map(price_range_labels)

data.head()

,restaurantName,address,city,region,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,O Me O Il Mare,Via Roma 45/47,Gragnano,Campania,80054,Italy,Luxury,"Italian Contemporary, Modern Cuisine","Known around the world as the town of pasta, G...","['Air conditioning', 'Interesting wine list', ...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 081 620 0550,http://omeoilmare.com
1,Donevandro,via Garibaldi 2,Popoli,Abruzzo,65026,Italy,Affordable,"Contemporary, Seasonal Cuisine","Up until a few years ago, the owner-chef at th...",['Air conditioning'],"['Mastercard', 'Visa']",+39 388 887 6858,http://www.donevandroristorante.it
2,Ape Vino e Cucina,Piazza Risorgimento 3,Alba,Piedmont,12051,Italy,Affordable,"Piedmontese, Contemporary",This attractive restaurant in the heart of Alb...,"['Air conditioning', 'Terrace', 'Wheelchair ac...","['Amex', 'Dinersclub', 'Maestrocard', 'Masterc...",+39 0173 363453,https://www.apewinebar.it/alba/
3,Da Bob Cook Fish,largo Parsano vecchio 16,Sorrento,Campania,80067,Italy,Affordable,Seafood,Working in partnership with the nearby fishmon...,"['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 081 1778 3873,https://www.dabobcookfish.com/
4,DA_MÓ,Via Bruno Buozzi 20,Matera,Basilicata,75100,Italy,Affordable,"Regional Cuisine, Contemporary","This new, restored restaurant in the upper par...","['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0835 686548,https://www.damoristorante.it/


## Interactive search interface
**Run the code cell to display the interactive restaurant search interface.**

---

## How to use:
Use dropdowns and checkboxes to filter restaurants by criteria like location, price, and cuisine.
Results are shown in an HTML table format.

---
Ensure ipywidgets is installed and enabled for full functionality.

In [ ]:
# Use the Advanced_Search_Engine function
sef.Advanced_Search_Engine(data)

In [ ]:
# Read the HTML content and display it
with open('filtered_results.html', 'r') as file:
    html_content = file.read()
    display(HTML(html_content))

Restaurant Name,Address,City,Website,cuisineType,creditCards,Services
Antica Pesa,via Garibaldi 18,Rome,https://www.anticapesa.it/,"Roman, Cuisine from Lazio","[Amex, Dinersclub, Jcb, Maestrocard, Mastercard, Visa]","[Air conditioning, Interesting wine list, Terrace]"


# Algorithmic Question (AQ)

<img src="https://github.com/Heibattttt/Michelin-restaurant-in-Italy-web-scraping/blob/main/Images/Algorithm%20question.png?raw=true" width=400>

A robot in a warehouse represented by a coordinate grid starts at (0, 0) and must collect *n* packages located at (x<sub>i</sub>, y<sub>i</sub>), where no two packages share the same coordinates, and (0, 0) is empty. The robot can move up ('U') or right ('R'), transitioning from (x, y) to (x+1, y) or (x, y+1). The task is to collect all *n* packages in the fewest moves, following the lexicographically smallest path when multiple shortest paths exist.


## Pseudocode of an algorithm that solves this problem

The `list_merge` function merges two sorted lists `A1` and `A2` into a single y-sorted list.

```
def list_merge(A1, A2):
    A = []
    i, j = 0, 0
    while i < len(A1) and j < len(A2):
        if (A1[i].x < A2[j].x) or (A1[i].x == A2[j].x and A1[i].y <= A2[j].y):
            A.append(A1[i])
            i += 1
        else:
            A.append(A2[j])
            j += 1
    while i < len(A1):
        A.append(A1[i])
        i += 1
    while j < len(A2):
        A.append(A2[j])
        j += 1
    return A
```
The `Merge sort` function sorts the list `A` with respect to x
```
def MergeSort(A):
    if len(A) <= 1:
        return A
    n = len(A)
    A1 = A[:n//2]
    A2 = A[n//2:]
    A1 = MergeSort(A1)
    A2 = MergeSort(A2)
    A = list_merge(A1, A2)
    return A
```
The main funcion is used to find the best path
```
t = number of test cases

def robot_path(t):
  for i in range(t):
    n = number of packages
    packages = list of coordinates of each package stored in tuple format

    packages = MergeSort(MergeSort(A))

    //Once we have the list sorted, we need to check if a valid path exists that respects the movement rules (only right and up movements).

    for j in range(n):
      if packages[j].y < packages[j-1].y:
        results.append("NO")
        break
      else:
        x' = 0 , y' = 0
        for (x, y) in packages:
            R = x - x'
            U = y - Y'
            path += "R" * R + "U" * U
            x' = x , y' = y
        results.append("YES " + path)

    return results
```


## Poof of the correctnes.

Induction on n:

* base case: n=1 ✓

It's obvious that for n=1 it works because When there is one package, the robot starts at (0,0) and moves directly to the package's position
(x1 , y1).
* induction hypothesis:
Assume the algorithm works for any set of k packages.

Now consider k+1 packages:

The algorithm sorts the packages by their x-coordinate and  by y-coordinate in case of ties.
After sorting the first k packages, the path formed by the robot is valid and lexicographically smallest by the inductive hypothesis.
We now show that adding the k+1-th package does not violate the rules and maintains lexicographical order.
Sorting ensures that the robot moves from smaller coordinates to larger ones and the validation step ensures that the moves are right or up.
By induction, we conclude that:
The algorithm constructs a valid path as it only moves right or up.
The path is lexicographically smallest due to the sorting order.
Therefore, the algorithm is correct and always produces the shortest valid path, and when there are multiple shortest paths, it produces the lexicographically smallest one.




## Time complexity

In this section, we analyze the time complexity of the proposed algorithm, considering each function individually to assess the overall efficiency.
The sorting step uses Merge Sort, which has a time complexity of O(nlogn).



#### **1. Time Complexity of `list_merge`**

The `list_merge` function performs a merge operation on two sorted lists (`A1` and `A2`) and combines them into a single sorted list `A`.

- We traverse both lists `A1` and `A2` once, comparing their elements and appending them to the result list `A`.
- The time complexity of merging two sorted lists of sizes n1 and n2 is \( O(n1 + n2 )\).
- Since we iterate over the entire length of both lists, the time complexity of `list_merge` is **O(n)**.

#### **2. Time Complexity of `MergeSort`**

The `MergeSort` function is a divide-and-conquer algorithm that recursively splits the list into halves, sorts them, and merges the sorted halves using `last_merge`.

- At each level of recursion, the list is divided into two halves, and we perform a merge at each level.
- At the first level, we merge two halves, each of size \( n/2 \), at the second level, we merge four sublists of size \( n/4 \), and so on, until the list size becomes 1.
- Each merge step takes **O(n)** time, and since we divide the list into halves at each recursion, there are \( \log n \) levels of recursion.

Thus, the time complexity of `MergeSort` is:
O(n log n)


#### **Final Time Complexity**

For \( t \) test cases, the overall time complexity of the entire algorithm is: O(t * n log n)

Since \( t \) has an upper bound of 10 and \( n \)  has an upper bound of 100, in the worst-case scenario, the algorithm's time complexity becomes:

$
O(t \cdot n \log n) \leq O(10 \cdot 100 \cdot \log 100) = O(1000 \cdot \log 100)
$




## Ask an LLM tool (such as ChatGPT, Claude AI, Gemini, Perplexity, etc.) to evaluate the time complexity of your code using Big O notation. Is the assessment accurate? If it differs from your previous analysis, which would be correct? Please explain your reasoning.

The assessment is accurate he two analyses yield the same outcome. However, mine includes a consideration regarding the maximum values of t and n.

## Assume now that the robot can also move towards the left or downwards, and consider the greedy approach: from the current location go to the closest package. Notice that now we can always collect all packages. Prove that the greedy algorithm is optimal (i.e., it minimizes the total distance traveled), or provide a counterexample showing that it is not.

The Greedy Algorithm Does Not Guarantee Optimality in Total Distance Traveled

#### **Counterexample**

Consider the following packages located at:

- $ (2, 2) $
- $ (5, 1) $
- $ (1, 5) $

The robot starts at $ (0, 0) $.

### Greedy Algorithm Path

The greedy algorithm picks the closest package at each step.

1. From $(0, 0)$, the robot moves to the closest package, which is $(2, 2)$.
The distance traveled is:
$$
\text{Distance} = (2 - 0)^2 + (2 - 0)^2 = 4 + 4 = 8 \quad \Rightarrow \quad \sqrt{8} \approx 2.83
$$

2. From $(2, 2)$, the closest package is $(5, 1)$, since the distance is:

$$
\text{Distance} = (5 - 2)^2 + (1 - 2)^2 = 9 + 1 = 10 \quad \Rightarrow \quad \sqrt{10} \approx 3.16
$$

Finally, the robot moves from $(5, 1)$ to $(1, 5)$, and the distance is:

$$
\text{Distance} = (1 - 5)^2 + (5 - 1)^2 = 16 + 16 = 32 \quad \Rightarrow \quad \sqrt{32} \approx 5.66
$$

**Total distance (Greedy):**
$$
2.83 + 3.16 + 5.66 = 11.65
$$

---

### Optimal Path

The optimal solution, considering all the packages as a whole, is to visit the packages in this order:

From $(0, 0)$ to $(1, 5)$:

$$
\text{Distance} = (1 - 0)^2 + (5 - 0)^2 = 1 + 25 = 26 \quad \Rightarrow \quad \sqrt{26} \approx 5.10
$$

From $(1, 5)$ to $(2, 2):

$$
\text{Distance} = (2 - 1)^2 + (2 - 5)^2 = 1 + 9 = 10 \quad \Rightarrow \quad \sqrt{10} \approx 3.16
$$

From $(2, 2)$ to $(5, 1):

$$
\text{Distance} = (5 - 2)^2 + (1 - 2)^2 = 9 + 1 = 10 \quad \Rightarrow \quad \sqrt{10} \approx 3.16
$$

**Total distance (Optimal):**
$$
5.10 + 3.16 + 3.16 = 11.42
$$

#### **Conclusion**

The greedy algorithm is **not optimal** for minimizing total distance traveled because it only considers the immediate closest package without accounting for the global arrangement of all packages.
